In [20]:
# Part I. ETL Pipeline for Pre-Processing the Files

### Changes in Submission 2
 - Add userId to song_subscription PRIMARY KEY
 - Add more commentary to this notebook

---
## Project Requirement
#### We are provided with a `music app history`  dataset - a list of csv files in the event_data root directory.
##### <font color='green'>Each csv file contains the following columns:</font>
- artist
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId


##### <font color='green'>Our goal is to create a Cassandra Schema that answers the following 3 queries::</font>
- QUERY 1 :: Select the artist, song title and song’s length in the music app history that was heard during sessionId = 338, and itemInSession = 4
- QUERY 2 :: Select ame of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
- QUERY 3 :: Select the every user name (first and last) in my music app history who listened to the song ‘All Hands Against His Own’

<br>


---
## Solution

##### <font color='green'>We'll create the following Cassandra tables</font>


- QUERY 1
    - <font color='gray'>Requirement: <i>Select the artist, song title and song’s length in the music app history that was heard during sessionId = 338, and itemInSession = 4</i></font>
    - Solution:
        - Create a `songplay_sessions` table composite primary key ((sessionId, itemInSession)) because we need to query for both of those columns

<br>

- QUERY 2
    - <font color='gray'>Requirement: <i>Select ame of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182</i></font>
    - Solution:
        - create a `user_sessions` table with PRIMARY KEY((userId, sessionId), itemInSession) because we need to query on both userId, and sessionId.
        - we add the custer key of itemInSession so that the results will be sorted by this column.

<br>

- QUERY 3
    - <font color='gray'>Requirement: <i>Select the every user name (first and last) in my music app history who listened to the song ‘All Hands Against His Own’</i></font>
    - Solution:
        - create a `song_subscribers` table with PRIMARY KEY (song, userId) because we need to query on the partition key "song",
        - we also want to make sure that the song+user pair is unique, so if both Bill and Marge are both subscribing to the same song, they will create two separate keys

<br>

##### <font color='green'>We do this in 6 steps in this Notebook</font>
- STEP 1 - CONNECT TO A LOCAL CASSANDRA CLUSTER AND CREATE AN `EVENT_DATA` KEYSPACE
- STEP 2 - SETUP A CLASS TO MANAGE THE POSITION-BASED VALUES ARRAY
- STEP 3 - SETUP FUNCTIONS THAT WE WILL USE LATER
- STEP 4 - LOAD DATA INTO CASSANDRA
- STEP 5 - PERFORM OUR QUERIES
- STEP 6 - CLEAN UP

In [21]:
# Import Python packages 
import csv
import glob
import os
import cassandra
from typing import List
import sql_queries

---
### STEP 1 - CONNECT TO A LOCAL CASSANDRA CLUSTER AND CREATE AN `EVENT_DATA` KEYSPACE

In [22]:
from cassandra.cluster import Cluster, Session

cluster = Cluster(['127.0.0.1'])
cassandra_session = cluster.connect()

# create and use schema
cassandra_session.execute("""
            CREATE KEYSPACE IF NOT EXISTS event_data
            WITH REPLICATION =
            { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

cassandra_session.set_keyspace('event_data')

print("connected to cassandra")

connected to cassandra


---
### STEP 2 - SETUP A CLASS TO MANAGE THE POSITION-BASED VALUES ARRAY

In [23]:
class CsvRecord:
    """
    utility class to make transform from csv to columns clearer
    """
    good_record: bool = False  # always false unless all fields were found

    def __init__(self, line : List[any]):
        """
        transform one values array into named columns    
        :param 
            line: array of 11 values
        """
        if len(line) != 11:
            print(f"bad line: {','.join(line)}")
            return
        self.artist = line[0]
        self.first_name= line[1]
        self.gender = line[2]
        self.item_in_session = int(line[3])
        self.last_name= line[4]
        self.song_length = float(line[5])
        self.level = line[6]
        self.location= line[7]
        self.session_id= int(line[8])
        self.song = line[9]
        self.user_id = int(line[10])
        self.good_record = True
                  
print("class created")

class created


---
### STEP 3 - SETUP FUNCTIONS THAT WE WILL USE LATER

##### <font color='green'>define a function that creates a list of files from the "event_data" directory</font>

In [24]:
def get_files(event_data: str) -> List[str]:
    """
    walk root directory and get all csv files in all subdirectories  
    
    :param 
        event_data: the root directory name to walk
    
    :return
        A list of all csv files in all subdirectories
    """

    # checking your current working directory
    print(f"Getting csv files from {os.getcwd()}")

    # Get your current folder and subfolder event data
    filepath = os.getcwd() + '/' + event_data

    # Create a for loop to create a list of files and collect each filepath
    for root, dirs, files in os.walk(filepath):
        # join the file path and roots with the subdirectories using glob
        file_path_list = glob.glob(os.path.join(root, '*.csv'))

    return file_path_list

print("method get_files created")

method get_files created


##### <font color='green'>define a function that uses the list of files and returns a single aggregate 'event_datafile_new.csv' file with all rows from all tables</font>

In [25]:
def merge_csv_data(file_path_list: List[str]) -> str:
    """
    merge a list of csv files into a single file

    :param
        file_path_list: a list of csv path names

    :return
        the merged single fle name (hard coded to event_datafile_new.csv)
    """

    # initiating an empty list of rows that will be generated from each file
    destination_file = 'event_datafile_new.csv'
    full_data_rows_list = []

    # for every filepath in the file path list dump to a massive array
    for f in file_path_list:

        # reading csv file
        with open(f, 'r', encoding='utf8', newline='') as csvfile:
            # creating a csv reader object
            csvreader = csv.reader(csvfile)
            next(csvreader)

            # extracting each data row one by one and append it
            for line in csvreader:
                full_data_rows_list.append(line)

    csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

    dumped_count = 0
    hits_count = 0

    # write the array into a new master file
    with open(destination_file, 'w', encoding='utf8', newline='') as f:
        writer = csv.writer(f, dialect='myDialect')
        writer.writerow(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
                         'level', 'location', 'sessionId', 'song', 'userId'])
        for row in full_data_rows_list:
            if row[0] == '':
                dumped_count += 1
                continue
            hits_count += 1
            writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

    print(f"dumped {dumped_count} records")
    print(f"kept {hits_count} records")

    # check the number of rows in your csv file
    with open(destination_file, 'r', encoding='utf8') as f:
        print(f"wrote {sum(1 for line in f)} lines to event_datafile_new.csv")

    return destination_file

print("method merge_csv_data created")

method merge_csv_data created


##### <font color='green'>define a function takes our new master file ('event_datafile_new.csv') and loads the data into our three Cassandra tables</font>
    For each line:
    - [x] insert (session_id, item_in_session, artist, song, song_length) into the `songplay_sessions` table
    - [x] insert (user_id, session_id, item_in_session, first_name, last_name, artist, song) into a `user_sessions` table
    - [x] insert (song, user_id, first_name, last_name) into a `song_subscribers` table

In [26]:
def load_csv_data(file: str, session: Session):
    """
    read the master csv file and load into cassadra tables  
    
    :param 
        file: the master csv file
        session: a cassandra.cluster.Session    
    """

    print("loading database...")
    line_count = 0

    # preconditions
    if not os.path.exists(file):
        raise (Exception(f"file named '{file}' was not found!!!"))
    if session is None:
        raise (Exception("Cassandra session is not set!!!!"))

    with open(file, encoding='utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader)  # skip header
        for line in csvreader:
            line_count += 1

            # read line array into a class instance
            r = CsvRecord(line)

            # song play sessions
            session.execute(sql_queries.insert_songplay_sessions, (
                r.session_id, r.item_in_session, r.artist, r.song, r.song_length
            ))

            # user sessions
            session.execute(sql_queries.insert_user_sessions, (
                r.user_id, r.session_id, r.item_in_session, r.first_name, r.last_name, r.artist, r.song
            ))
            # songs by user
            session.execute(sql_queries.insert_song_subscribers, (
                r.song, r.user_id, r.first_name, r.last_name
            ))
    print(f"loaded {line_count} lines ...")
    
print("method load_csv_data() created")

method load_csv_data() created


##### <font color='green'>define a function that creates our three Cassandra tables</font>

In [27]:
def database_init(session: Session):
    """
    create all tables in Cassandra 
    
    :param 
        session: a cassandra.cluster.Session    
    """

    print("initializing database...")
    from cassandra.cluster import Cluster
    try:

        drop_tables(session=session)

        # songplay sessions
        session.execute(sql_queries.create_songplay_sessions)

        # user_sessions
        session.execute(sql_queries.create_user_sessions)

        # song_subscribers
        session.execute(sql_queries.create_song_subscribers)
        return session

    except Exception as e:
        print(e)
        raise e
        
print("method database_init() created")

method database_init() created


##### <font color='green'>define a function that drops our tables</font>

In [28]:
def drop_tables(session: Session):
    """
    drop all tables in session (keyspace)

    :param
        session: a cassandra.cluster.Session
    """
    print("dropping database first...")
    from cassandra.cluster import Cluster
    try:
        session.execute(sql_queries.drop_songplay_sessions)
        session.execute(sql_queries.drop_user_sessions)
        session.execute(sql_queries.drop_song_subscribers)
        return session

    except Exception as e:
        print(e)
        raise e

print("method drop_tables() created")

method drop_tables() created


---
### Step 4 - LOAD DATA INTO CASSANDRA
Now that we have defined all classes and functions, we'll begin loading and querying our data


##### <font color='green'>First, we'll use the function we created above to create our three Cassandra tables</font>

In [29]:
# create database cluster/schema and return session
database_init(session=cassandra_session)

initializing database...
dropping database first...



##### <font color='green'>Next, we'll read all files from the event_data root directory and combine them into a single master_file</font>

In [30]:
# read all files in the event_data root directory
csv_files = get_files(event_data="event_data")

# combine them into a single file
master_file = merge_csv_data(file_path_list=csv_files)


Getting csv files from /Users/christopherlomeli/Source/courses/udacity-cassandra-model
dumped 1131 records
kept 6171 records
wrote 6172 lines to event_datafile_new.csv



##### <font color='green'>Finally, we'll use the master_file to load data into our 3 Cassandra tables</font>

In [31]:
# load data into cassandra from the master file
load_csv_data(master_file, cassandra_session)

loading database...
loaded 6171 lines ...


<br>


---
#### Step 5 - PERFORM OUR QUERIES
Now we are ready to test our solution by running the required 3 queries

<br>

##### <font color='green'>5.1 - Run some Sanity tests - just verify that tables have data by running a select count(*) for each table</font>

In [32]:
print("\n---- Count table rows ---")

rows = cassandra_session.execute("select count(*) from songplay_sessions")
for row in rows:
    print(f"songplay_sessions has {row.count} rows")

rows = cassandra_session.execute("select count(*) from user_sessions")
for row in rows:
    print(f"user_sessions has {row.count} rows")

rows = cassandra_session.execute("select count(*) from song_subscribers")
for row in rows:
    print(f"song_subscribers has {row.count} rows")




---- Count table rows ---
songplay_sessions has 6171 rows
user_sessions has 6171 rows
song_subscribers has 6001 rows


##### <font color='green'>5.2 - Create Query functions so that we can call each query with different arguments</font>

In [33]:
# QUERY1 :: Define a function to Select the artist, song title and song’s length in the music app history that was heard durin
def query_songplays(session: Session, session_id : int, ordinal: int ):
    print(f"\n---- results for songplays with sessionId={session_id}, and item={ordinal} ----")
    rows = session.execute(sql_queries.select_songplay_sessions, (session_id, ordinal))
    for row in rows:
        print(row)


# QUERY2 :: Define a function to Select the artist, song title and song’s length in the music app history that was heard durin
def query_user_sessions(session: Session, user_id : int, session_id : int):
    print(f"\n---- results for user sessions with userId={user_id} and sessionId={session_id} ----")
    rows = session.execute(sql_queries.select_user_sessions, (user_id, session_id))
    for row in rows:
        print(row)


# QUERY 3 ::
def query_song_subscribers(session: Session, song: str):
    print(f"\n---- results for all users listening to '{song} ----")
    rows = session.execute(sql_queries.select_song_subscribers, (song,))
    for row in rows:
        print(row)

print("created queries methods...")

created queries methods...



##### <font color='green'>5.3 Now, run the queries</font>

<font color='blue'>QUERY1 - Select the artist, song title and song’s length in the music app history that was heard during sessionId = 338, and itemInSession = 4</font>

In [34]:
query_songplays(session=cassandra_session, session_id=338, ordinal=4)


---- results for songplays with sessionId=338, and item=4 ----
Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', song_length=495.3073)


<font color='blue'>QUERY2 - Select name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182</font>

In [35]:
query_user_sessions(session=cassandra_session, user_id=10, session_id=182)


---- results for user sessions with userId=10 and sessionId=182 ----
Row(artist='Down To The Bone', song="Keep On Keepin' On", iteminsession=0, firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', iteminsession=1, firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', iteminsession=2, firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', iteminsession=3, firstname='Sylvie', lastname='Cruz')


<font color='blue'>QUERY3 - Select every user name (first and last) in my music app history who listened to the song ‘All Hands Against His Own’</font>

In [36]:
query_song_subscribers(session=cassandra_session, song="All Hands Against His Own")


---- results for all users listening to 'All Hands Against His Own ----
Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


<br>

---
#### Step 6 - CLEAN UP

In [37]:
## TO-DO: Drop the table before closing out the sessions
# drop_tables(session=cassandra_session)
# cassandra_session.execute("drop keyspace if exists event_data")

In [38]:
cassandra_session.shutdown()
cluster.shutdown()